In [ ]:
!pip install requests PyPDF2 gdown
!pip install 'markitdown[pdf]'
!pip install langchain_mcp_adapters langchain_google_genai langchain-openai

In [7]:
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('zhao')

In [8]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",
                             api_key=GEMINI_VERTEX_API_KEY,
                             vertexai=True,
                             temperature=0)

In [ ]:
import os
import gdown

folder_id = "1adYKq7gSSczFP3iikfA8Er-HSZP6VM7D"
folder_url = f"https://drive.google.com/drive/folders/{folder_id}"

output_dir = "downloaded_cvs"
os.makedirs(output_dir, exist_ok=True)

gdown.download_folder(
    url=folder_url,
    output=output_dir,
    quiet=False,
    use_cookies=False
)

In [ ]:
# =====================================================
#  Load and display all CV PDFs in order
# =====================================================
import os
from markitdown import MarkItDown

cv_dir = "downloaded_cvs"

# Initialize MarkItDown
md = MarkItDown(enable_plugins=False)

# Collect and sort PDFs numerically
pdf_files = sorted(
    [f for f in os.listdir(cv_dir) if f.lower().endswith(".pdf")],
    key=lambda x: int("".join(filter(str.isdigit, x)))  # CV_1.pdf → 1
)

all_cvs = []

for pdf_name in pdf_files:
    pdf_path = os.path.join(cv_dir, pdf_name)
    result = md.convert(pdf_path)

    all_cvs.append({
        "file": pdf_name,
        "text": result.text_content
    })

    print("=" * 80)
    print(f"📄 {pdf_name}")
    print("=" * 80)
    print(result.text_content)
    print("\n\n")


In [ ]:
import asyncio
import json
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient({
    "social_graph": {
        "transport": "http",
        "url": "https://ftec5660.ngrok.app/mcp",
        "headers": {"ngrok-skip-browser-warning": "true"}
    }
})

mcp_tools = await client.get_tools()
for tool in mcp_tools:
    print(tool.name)
    print(tool.description)
    print(tool.args)
    print("\n\n------------------------------------------------------\n\n")

In [ ]:
def extract_structured_info(cv_text):
    prompt = f"""
从以下简历中提取结构化信息，严格按照示例格式返回JSON，不要用代码块包裹，不要任何额外解释：
{{
  "name": "候选人全名",
  "location": "城市/国家",
  "work_experience": [
    {{
      "company": "公司名",
      "title": "职位",
      "start_year": 开始年份,
      "end_year": 结束年份或null
    }}
  ],
  "education": [
    {{
      "school": "学校名",
      "degree": "学位",
      "field": "专业",
      "start_year": 开始年份,
      "end_year": 结束年份
    }}
  ]
}}

简历文本：
{cv_text}
"""
    response = llm.invoke(prompt)

    # 清洗和解析
    try:
        return json.loads(response.content.strip())
    except json.JSONDecodeError:
        cleaned = response.content.strip().replace("```json", "").replace("```", "")
        try:
            return json.loads(cleaned)
        except:
            print(f"解析失败，原始返回：{response.content}")
            return {}

# 3. 批量提取并打印结果
structured_cvs = []
for cv in all_cvs:
    print(f"正在提取结构化信息：{cv['file']}")
    structured_data = extract_structured_info(cv["text"])
    structured_cvs.append({
        "file": cv["file"],
        "structured": structured_data
    })

# 打印结果
for cv in structured_cvs:
    print(f"\n=== {cv['file']} ===")
    print(json.dumps(cv["structured"], indent=2))

In [ ]:
import asyncio
from langchain_mcp_adapters.client import MultiServerMCPClient

async def connect_mcp():

    client = MultiServerMCPClient({
        "social_graph": {
            "transport": "http",
            "url": "https://ftec5660.ngrok.app/mcp",
            "headers": {"ngrok-skip-browser-warning": "true"}
        }
    })


    try:
        tools = await client.get_tools()
        print("✅ MCP 服务器连接成功！")
        print("可用工具：")
        for tool in tools:
            print(f"- {tool.name}: {tool.description}")
        return client
    except Exception as e:
        print(f"❌ 连接失败: {type(e).__name__}: {str(e)}")
        return None


mcp_client = await connect_mcp()

In [ ]:
async def test_search_with_session():

    if mcp_client is None:
        print("❌ 客户端未连接，无法创建会话")
        return


    first_cv = structured_cvs[0]
    name = first_cv["structured"]["name"]


    try:
        async with mcp_client.session("social_graph") as session:
            print(f"🔎 正在搜索: {name}")

            result = await session.call_tool(
                name="search_facebook_users",
                arguments={
                    "q": name,
                    "limit": 1,
                    "fuzzy": True
                }
            )
            print("✅ 搜索结果:", result)
            return result
    except Exception as e:
        print(f"❌ 会话调用失败: {type(e).__name__}: {str(e)}")


await test_search_with_session()

In [ ]:
async def get_user_profile():

    user_id = 8

    try:
        async with mcp_client.session("social_graph") as session:
            print(f"👤 正在获取用户 ID {user_id} 的资料...")
            profile = await session.call_tool(
                name="get_facebook_profile",
                arguments={"user_id": user_id}
            )
            print("✅ 用户资料获取成功！")
            print(profile)
            return profile
    except Exception as e:
        print(f"❌ 获取用户资料失败: {type(e).__name__}: {str(e)}")


user_profile = await get_user_profile()

In [ ]:
import json

async def get_first_user_profile_final():
    first_cv = structured_cvs[0]
    candidate_name = first_cv["structured"]["name"]


    async with mcp_client.session("social_graph") as session:
        raw_search = await session.call_tool(
            name="search_facebook_users",
            arguments={"q": candidate_name, "limit": 1, "fuzzy": True}
        )


    json_str = raw_search.content[0].text
    search_result_data = json.loads(json_str)

    if not search_result_data:
        print("❌ 没找到用户")
        return None

    user_id = search_result_data[0]["id"]
    print(f"✅ 找到用户 ID: {user_id}")


    async with mcp_client.session("social_graph") as session:
        raw_profile = await session.call_tool(
            name="get_facebook_profile",
            arguments={"user_id": user_id}
        )


    profile_json_str = raw_profile.content[0].text
    profile_data = json.loads(profile_json_str)

    print("✅ 用户资料解析成功：")
    print(json.dumps(profile_data, indent=2, ensure_ascii=False))
    return profile_data


user_profile = await get_first_user_profile_final()

In [ ]:
def detect_first_cv_discrepancies():
    cv_data = structured_cvs[0]["structured"]

    discrepancies = []


    cv_school = cv_data["education"][0]["school"] if cv_data.get("education") else "N/A"
    social_edu = user_profile.get("education", "N/A")
    if cv_school not in social_edu:
        discrepancies.append({
            "field": "education",
            "cv": cv_school,
            "social": social_edu,
            "issue": "学校不匹配"
        })


    cv_company = cv_data["work_experience"][0]["company"] if cv_data.get("work_experience") else "N/A"
    social_company = user_profile.get("current_company", "N/A")
    if cv_company not in social_company:
        discrepancies.append({
            "field": "current_company",
            "cv": cv_company,
            "social": social_company,
            "issue": "公司不匹配"
        })

    print("🔍 差异检测结果:")
    if discrepancies:
        for d in discrepancies:
            print(f"- {d['issue']}: CV 声称 '{d['cv']}', 社交显示 '{d['social']}'")
    else:
        print("✅ 未发现差异")
    return discrepancies


discrepancies = detect_first_cv_discrepancies()

In [ ]:
def generate_first_report():
    cv_file = structured_cvs[0]["file"]
    cv_name = structured_cvs[0]["structured"]["name"]
    social_name = user_profile.get("display_name", "N/A")

    report = f"""
# 📋 CV 验证报告 - {cv_file}
## 基本信息
- 候选人姓名: {cv_name}
- 匹配社交账号: {social_name}

## 🔍 验证结果
"""
    if discrepancies:
        report += f"### ⚠️ 发现 {len(discrepancies)} 处差异:\n"
        for d in discrepancies:
            report += f"- {d['issue']}: CV 声称 '{d['cv']}', 社交显示 '{d['social']}'\n"
    else:
        report += "### ✅ 未发现明显差异，简历信息初步验证通过。\n"

    print(report)
    return report


first_report = generate_first_report()

In [25]:
import asyncio
import json

async def verify_all_cvs():
    if mcp_client is None:
        print("❌ MCP 客户端未连接")
        return
    if len(structured_cvs) == 0:
        print("❌ 没有结构化简历数据")
        return

    final_results = []

    for idx, cv in enumerate(structured_cvs, 1):
        cv_file = cv["file"]
        cv_data = cv["structured"]
        candidate_name = cv_data.get("name", "").strip()
        print(f"\n{'='*60}")
        print(f"【{idx}/{len(structured_cvs)}】正在处理: {cv_file} | 姓名: {candidate_name}")
        print('='*60)


        try:
            async with mcp_client.session("social_graph") as session:
                raw_search = await session.call_tool(
                    name="search_facebook_users",
                    arguments={"q": candidate_name, "limit": 1, "fuzzy": True}
                )

            search_result_data = json.loads(raw_search.content[0].text)
            if not search_result_data:
                print(f"⚠️ 未找到匹配用户，跳过 {cv_file}")
                final_results.append({
                    "file": cv_file,
                    "status": "skipped",
                    "reason": "no_matching_users"
                })
                continue
            best_user = search_result_data[0]
            user_id = best_user["id"]
            print(f"✅ 找到匹配用户: ID={user_id}, 姓名={best_user['display_name']}")
        except Exception as e:
            print(f"❌ 搜索失败: {str(e)}")
            final_results.append({
                "file": cv_file,
                "status": "failed",
                "stage": "search",
                "error": str(e)
            })
            continue


        try:
            async with mcp_client.session("social_graph") as session:
                raw_profile = await session.call_tool(
                    name="get_facebook_profile",
                    arguments={"user_id": user_id}
                )

            profile_data = json.loads(raw_profile.content[0].text)
            if not profile_data:
                print(f"⚠️ 用户资料为空，跳过 {cv_file}")
                final_results.append({
                    "file": cv_file,
                    "status": "skipped",
                    "reason": "empty_profile"
                })
                continue
            print("✅ 用户资料获取成功")
        except Exception as e:
            print(f"❌ 获取资料失败: {str(e)}")
            final_results.append({
                "file": cv_file,
                "status": "failed",
                "stage": "get_profile",
                "error": str(e)
            })
            continue


        discrepancies = []

        if profile_data.get("display_name") and cv_data.get("name"):
            if not any(part.lower() in profile_data["display_name"].lower() for part in cv_data["name"].split()):
                discrepancies.append({
                    "category": "Identity",
                    "cv_claim": cv_data["name"],
                    "social_evidence": profile_data["display_name"],
                    "issue": "姓名不匹配"
                })

        cv_schools = [edu.get("school", "") for edu in cv_data.get("education", [])]
        social_edu = profile_data.get("education", "")
        for school in cv_schools:
            if school and school not in social_edu:
                discrepancies.append({
                    "category": "Education",
                    "cv_claim": school,
                    "social_evidence": social_edu if social_edu else "未填写",
                    "issue": "简历中的学校未在社交资料中体现"
                })

        cv_companies = [exp.get("company", "") for exp in cv_data.get("work_experience", [])]
        social_company = profile_data.get("current_company", "")
        for company in cv_companies:
            if company and social_company and company not in social_company:
                discrepancies.append({
                    "category": "Employment",
                    "cv_claim": company,
                    "social_evidence": social_company,
                    "issue": "简历中的公司与社交资料当前公司不符"
                })


        report = f"# 📋 CV 验证报告 - {cv_file}\n"
        report += f"## 基本信息\n- 候选人姓名: {cv_data.get('name', 'N/A')}\n"
        report += f"- 匹配社交账号: {profile_data.get('display_name', '未找到')}\n"
        if discrepancies:
            report += f"## 🔍 发现 {len(discrepancies)} 处差异:\n"
            for i, d in enumerate(discrepancies, 1):
                report += f"{i}. **[{d['category']}] {d['issue']}**\n   - 简历: {d['cv_claim']}\n   - 社交: {d['social_evidence']}\n"
        else:
            report += "## 🔍 未发现明显差异，简历信息初步验证通过。\n"
        print(report)

        final_results.append({
            "file": cv_file,
            "status": "completed",
            "cv_data": cv_data,
            "social_data": profile_data,
            "discrepancies": discrepancies,
            "report": report
        })


    with open("CV_Verification_Final_Report.json", "w", encoding="utf-8") as f:
        json.dump(final_results, f, ensure_ascii=False, indent=2)
    print(f"\n🎉 全部处理完成！共处理 {len(final_results)} 份简历")
    print("📄 完整报告已保存为: CV_Verification_Final_Report.json")


await verify_all_cvs()


【1/5】正在处理: CV_1.pdf | 姓名: John Smith
✅ 找到匹配用户: ID=8, 姓名=John Smith
✅ 用户资料获取成功
# 📋 CV 验证报告 - CV_1.pdf
## 基本信息
- 候选人姓名: John Smith
- 匹配社交账号: John Smith
## 🔍 发现 2 处差异:
1. **[Education] 简历中的学校未在社交资料中体现**
   - 简历: McGill University
   - 社交: Doctoral Degree
2. **[Employment] 简历中的公司与社交资料当前公司不符**
   - 简历: ByteDance
   - 社交: Hang Seng Bank


【2/5】正在处理: CV_2.pdf | 姓名: Minh Pham
✅ 找到匹配用户: ID=62, 姓名=Minh Pham
✅ 用户资料获取成功
# 📋 CV 验证报告 - CV_2.pdf
## 基本信息
- 候选人姓名: Minh Pham
- 匹配社交账号: Minh Pham
## 🔍 发现 3 处差异:
1. **[Education] 简历中的学校未在社交资料中体现**
   - 简历: The University of Hong Kong
   - 社交: Master's Degree
2. **[Employment] 简历中的公司与社交资料当前公司不符**
   - 简历: BCG
   - 社交: Deloitte
3. **[Employment] 简历中的公司与社交资料当前公司不符**
   - 简历: Tencent
   - 社交: Deloitte


【3/5】正在处理: CV_3.pdf | 姓名: Wei Zhang
✅ 找到匹配用户: ID=76, 姓名=Wei Zhang
✅ 用户资料获取成功
# 📋 CV 验证报告 - CV_3.pdf
## 基本信息
- 候选人姓名: Wei Zhang
- 匹配社交账号: Wei Zhang
## 🔍 发现 2 处差异:
1. **[Education] 简历中的学校未在社交资料中体现**
   - 简历: University of Tokyo
   - 社交: Master's Degree
2. **[Empl

In [1]:
# =====================================================
#  Evaluation code
# =====================================================

def evaluate(scores, groundtruth, threshold=0.5):
    """
    scores: list of floats in [0, 1], length = 5
    groundtruth: list of ints (0 or 1), length = 5
    """
    assert len(scores) == 5
    assert len(groundtruth) == 5

    correct = 0
    decisions = []

    for s, gt in zip(scores, groundtruth):
        pred = 1 if s > threshold else 0
        decisions.append(pred)
        if pred == gt:
            correct += 1

    final_score = correct / len(scores)

    return {
        "decisions": decisions,
        "correct": correct,
        "total": len(scores),
        "final_score": final_score
    }


In [3]:
scores = [0.2, 0.3, 0.4, 0.5, 0.6]# Your code should generate this list [0.2, 0.3, 0.4, 0.5, 0.6]
groundtruth = [1, 1, 1, 0, 0] # Do not modify

result = evaluate(scores, groundtruth)
print(result)


{'decisions': [0, 0, 0, 0, 1], 'correct': 1, 'total': 5, 'final_score': 0.2}
